In [ ]:
from datasets import load_dataset
import pandas as pd

In [ ]:
data = load_dataset("NeuroDragon/BuggedPythonLeetCode")
df = pd.DataFrame(data["train"])
df = df[df["answer_status"] == "found"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train/bugged_leetcode_no_replaced.parque(…):   0%|          | 0.00/23.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14113 [00:00<?, ? examples/s]

In [ ]:
bugged = pd.DataFrame({"Code_text": df["bugged_code"], "label": 0})
fixed = pd.DataFrame({"Code_text": df["original_code"], "label": 1})
final_df = pd.concat([bugged, fixed])
final_df.shape

X = final_df["Code_text"]
y = final_df["label"]

In [ ]:
final_df.head()

,Code_text,label
0,"def twoSum(nums, target):\n map = {}\n f...",0
4,def reverse(x: int) -> int:\n sign = -1 if ...,0
5,"def myAtoi(s):\n result, i, sign = 0, 0, 1\...",0
6,def is_palindrome(x: int) -> bool:\n if x <...,0
7,"def is_match(s: str, p: str) -> bool:\n m, ...",0


In [ ]:
final_df.sample()

,Code_text,label
2455,def int_to_roman(num):\n romansromans = [\n...,0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

model = TfidfVectorizer(
      ngram_range=(1,3),
      max_features=100000,
      min_df=2
)

model.fit(X)
X_Tfidf = model.transform(X)



In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_Tfidf,y,test_size=0.2,random_state=42)

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression



model = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=42
)
model.fit(X_train,y_train)



RandomForestClassifier(max_depth=5, random_state=42)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(accuracy_score(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred))

0.6324894514767933
[[1019 1347]
 [ 395 1979]]
              precision    recall  f1-score   support

           0       0.72      0.43      0.54      2366
           1       0.60      0.83      0.69      2374

    accuracy                           0.63      4740
   macro avg       0.66      0.63      0.62      4740
weighted avg       0.66      0.63      0.62      4740



In [ ]:
final_df.shape

(23698, 2)

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate = 0.1,
    random_state=42
)

model.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(accuracy_score(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred))

0.6789029535864979
[[ 948 1418]
 [ 104 2270]]
              precision    recall  f1-score   support

           0       0.90      0.40      0.55      2366
           1       0.62      0.96      0.75      2374

    accuracy                           0.68      4740
   macro avg       0.76      0.68      0.65      4740
weighted avg       0.76      0.68      0.65      4740



In [ ]:
import re
import pandas as pd

feature = []

for i in final_df['Code_text']:
    feature.append({
        "for_count": len(re.findall(r'\bfor\b', i)),
        "if_count": len(re.findall(r'\bif\b', i)),
        "return_count": len(re.findall(r'\breturn\b', i)),
        "comment_count": len(re.findall(r'#', i)),
        "has_try_except": 1 if re.search(r'\btry\b', i) and re.search(r'\bexcept\b', i) else 0,
        "num_functions": len(re.findall(r'\bdef\b', i)),
        "avg_indent_depth": sum(len(line) - len(line.lstrip()) for line in i.split('\n')) / max(len(i.split('\n')), 1)
    })

feature_df = pd.DataFrame(feature)


In [ ]:
feature_df.head()

,for_count,if_count,return_count,comment_count,has_try_except,num_functions,avg_indent_depth
0,1,1,2,0,0,1,6.000000
1,0,1,1,0,0,1,4.444444
2,0,4,3,0,0,1,5.666667
3,0,1,2,0,0,1,5.000000
4,4,2,1,0,0,1,6.352941


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

model = TfidfVectorizer(
      ngram_range=(1,3),
      max_features=100000,
      min_df=2
)

model.fit(X)
X_Tfidf = model.transform(X)


In [ ]:
from scipy.sparse import hstack, csr_matrix

X_Tfidf = hstack((X_Tfidf, csr_matrix(feature_df)))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_Tfidf,y,test_size=0.2,random_state=42)

from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate = 0.1,
    random_state=42
)

model.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(accuracy_score(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred))

0.6782700421940928
[[ 937 1429]
 [  96 2278]]
              precision    recall  f1-score   support

           0       0.91      0.40      0.55      2366
           1       0.61      0.96      0.75      2374

    accuracy                           0.68      4740
   macro avg       0.76      0.68      0.65      4740
weighted avg       0.76      0.68      0.65      4740

